In [1]:
from sqlalchemy import create_engine
from configparser import ConfigParser
from tqdm.auto import tqdm
import uuid

In [2]:
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return db

In [5]:
def dump_3m(engine):
    file = uuid.uuid4()
    #select_stmt1 = f"COPY (DELETE FROM dataset WHERE sampleid IN (SELECT sampleid from dataset where status = 2 and language = 'en' LIMIT 4500000) RETURNING *) TO '/home/cah/clipped/ok-{file}.csv' DELIMITER '|' CSV HEADER;"
    select_stmt1 = f"COPY (DELETE FROM dataset WHERE sampleid IN (SELECT sampleid from dataset where status > 8 LIMIT 4500000) RETURNING *) TO '/home/cah/rejected/bad-{file}.csv' DELIMITER '|' CSV HEADER;"
    conn = engine.raw_connection()
    try:
        cur = conn.cursor()
        cur.execute(select_stmt1)
        conn.commit()
        cur.close()
    except:
        print("error")
    conn.close()
    return

In [7]:
params = config()
cycles = 24

engine = create_engine(f'postgresql://{params["user"]}:{params["password"]}@{params["host"]}:5432/{params["database"]}',pool_size=3, max_overflow=5, pool_pre_ping=True)

for i in tqdm(range(cycles)):
    dump_3m(engine)


  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
import pandas as pd
df = pd.read_csv("4a44ef0d-00ba-4842-b385-3d06245053d9.csv", delimiter="|")
df.to_parquet("4a44ef0d-00ba-4842-b385-3d06245053d9.parquet")